In [37]:
from tqdm import tqdm
from utils import create_resample, normalize
import numpy as np 
from numpy import asarray,zeros, array
import pandas as pd

import tensorflow as tf
import torch
from keras import backend as k
from keras.preprocessing.text import Tokenizer
from transformers import AutoTokenizer
from keras.utils import pad_sequences
from keras.layers import LSTM,Dropout,Bidirectional,Input,Embedding, Dense,Concatenate,Flatten, Multiply,Average,Reshape,Lambda
from keras.models import Model
from tensorflow.keras.layers import concatenate
from keras.utils import to_categorical
from keras import optimizers,regularizers
from keras.optimizers import Adam

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import random,statistics,json
from sklearn.model_selection import StratifiedKFold
from sklearn.utils import resample
import datetime

In [ ]:
def attentionScores(var):
    Q_t, K_t, V_t = var[0],var[1],var[2]
    scores = tf.matmul(Q_t, K_t, transpose_b=True)
    distribution = tf.nn.softmax(scores)
    scores = tf.matmul(distribution, V_t)
    return scores

In [ ]:
data = pd.read_csv("./data/climate_change_tweets.csv")
features = normalize(data[['tweet_id', 'prepContent', 'len_text', 'num_hashtag', 'num_emoji', 'num_mention', 
                           'degC', 'clsC', 'btwC', 'engage', 'pagerank']])
labels = data[['stance_label', 'vader_label']]

prep_id = features.tweet_id.values[i]
prep_text = features.prepContent.values[i]
prep_senti = labels.vader_label.values[i]
prep_stance = labels.stance_label.values[i]
prep_graph = data[['len_text', 'num_hashtag', 'num_emoji', 'num_mention', 
                          'degC', 'clsC', 'btwC', 'engage', 'pagerank']].to_numpy()

In [42]:
########### converting stance labels into categorical labels ########
label_encoder = LabelEncoder()
final_lbls = prep_stance
values = array(final_lbls)
total_integer_encoded = label_encoder.fit_transform(values)

prep_enc_stance = total_integer_encoded
total_integer_encoded = to_categorical(total_integer_encoded)

prep_stance = total_integer_encoded

########### converting senti labels into categorical labels ########
label_encoder = LabelEncoder()
final_lbls = prep_senti
values = array(final_lbls)
total_integer_encoded=label_encoder.fit_transform(values)

prep_enc_senti = total_integer_encoded
total_integer_encoded = to_categorical(total_integer_encoded)

prep_senti = total_integer_encoded

In [ ]:
MAX_SEQ = 150
MAX_LENGTH = 150

tokenizer = Tokenizer()
tokenizer.fit_on_texts(total_text)
total_sequence = tokenizer.texts_to_sequences(total_text)
padded_docs = pad_sequences(total_sequence, maxlen=MAX_SEQ, padding='post')

vocab_size = len(tokenizer.word_index) + 1

print("downloading {} embedding ...".format(EMB_TYPE))
embedding_matrix = np.load("./dataset/embed_matrix_glove.npy")

print("embedding matrix ****************",embedding_matrix.shape)
print("non zeros bert :", sum(np.all(embedding_matrix, axis=1)))

total_emb = np.load("./dataset/embed_node_gcn_20240121.npy")
total_graph = prep_graph
total_feat_sequence = np.concatenate((padded_docs, total_graph), axis=1)
total_feat_sequence = np.concatenate((total_feat_sequence, total_emb), axis=1)

emb_len = embedding_matrix.shape[1]
emb_feat_len = total_feat_sequence.shape[1]
print("input matrix ****************", total_feat_sequence.shape)

In [ ]:
total_labels_stance = np.array(prep_stance)
total_labels_senti = np.array(prep_senti)
total_sequence = np.array(padded_docs)

total_labels_stance_enc = np.argmax(total_labels_stance, axis=1)
list_acc_stance, list_acc_senti = [], []
list_f1_stance, list_f1_senti = [], []

kf = StratifiedKFold(n_splits=5, random_state=None,shuffle=False)
fold = 0
results = []
for train_index, test_index in kf.split(total_sequence, total_labels_stance_enc):
    print("K FOLD ::::::", fold)
    fold = fold + 1

    ############## Stance inputs #############
    FEAT_LENGTH = 9
    NODEEMB_LENGTH = 32
    
    W = tf.Variable(1.0 ,trainable=True)
    input1_stance = Input (shape = (MAX_LENGTH,)) # MAX_LENGTH = 150
    input2_stance = Input (shape=(FEAT_LENGTH,)) # FEAT_LENGTH = 9
    input3_stance = Input (shape=(NODEEMB_LENGTH,1)) # NODEEMB_LENGTH = 32
    
    input1_text_stance = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=150, name='text_embed_stance')(input1_stance) 
    input2_text_stance = Dense(50)(input2_stance) 
    
    lstm_stance = Bidirectional(LSTM(100, name='lstm_inp1_stance',activation='tanh',dropout=.2,kernel_regularizer=regularizers.l2(0.07)))(input1_text_stance)
    lstm_nodeemb = Bidirectional(LSTM(100, name='lstm_inp3_nodeemb',activation='tanh',dropout=.2,kernel_regularizer=regularizers.l2(0.07)))(input3_stance)
    text_final_stance = Dense(100, activation="relu")(lstm_stance)
    text_emb_stance = Dense(100, activation="relu")(lstm_nodeemb)
    text_final_stance = Concatenate()([text_final_stance, input2_text_stance, W*text_emb_stance])

    Q_t = Dense(100, activation="relu")(text_final_stance)
    K_t = Dense(100, activation="relu")(text_final_stance)
    V_t = Dense(100, activation="relu")(text_final_stance)
    IA_text_stance = Lambda(attentionScores)([Q_t,K_t,V_t])
    T1 = IA_text_stance
    
    ### capturing weights ######
    try_1 = layers.Dense(100, activation="relu")
    T1_try = try_1(T1)
    try_1_weights = try_1.get_weights()

    stance_specific_output = Dense(2, activation="softmax", name="task_specific_act")(T1_try)

    ############## Sentiment inputs #############
    W2 = tf.Variable(1.0 ,trainable=True)
    input1_senti = Input (shape = (MAX_LENGTH,))
    input2_senti = Input (shape=(FEAT_LENGTH,))
    input3_senti = Input (shape=(NODEEMB_LENGTH,1))
    
    input1_text_senti = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=150, name='text_embed_senti')(input1_senti) 
    input2_text_senti = Dense(50)(input2_senti) 

    lstm_senti = Bidirectional(LSTM(100, name='lstm_inp1_senti',activation='tanh',dropout=.2,kernel_regularizer=regularizers.l2(0.07)))(input1_text_senti)
    lstm_senti_nodeemb = Bidirectional(LSTM(100, name='lstm_inp3_sent_nodeemb',activation='tanh',dropout=.2,kernel_regularizer=regularizers.l2(0.07)))(input3_senti)
    text_final_senti = Dense(100, activation="relu")(lstm_senti)
    text_emb_senti = Dense(100, activation="relu")(lstm_senti_nodeemb)
    text_final_senti = Concatenate()([text_final_senti, input2_text_senti, W2*text_emb_senti])
    
    Q_t = Dense(100, activation="relu")(text_final_senti)
    K_t = Dense(100, activation="relu")(text_final_senti)
    V_t = Dense(100, activation="relu")(text_final_senti)
    IA_text_senti = Lambda(attentionScores)([Q_t,K_t,V_t])
    T2 = IA_text_senti
    try_2 = layers.Dense(100, activation="relu")
    T2_try = try_2(T2)
    try_2_weights = try_2.get_weights()

    senti_specific_output = Dense(3, activation="softmax", name="task_specific_senti")(T2_try)

    ################ Average of 3 tensors ##############
    M=Average()([T1_try,T2_try])
    print("M::::",M)

    ############# shared specific attention for stance ###############
    G_stance_layer = layers.Dense(100, activation="sigmoid")
    G_stance = G_stance_layer(M)
    G_stance_layer.set_weights(try_1_weights)
    G_stance_out = Multiply()([G_stance,M])

    Shared_query = Dense(100, activation="relu")(T1_try)
    Shared_key = Dense(100, activation="relu")(M)
    Shared_value = Dense(100, activation="relu")(M)
    Shared_stance_attn = Lambda(attentionScores)([Shared_query,Shared_key,Shared_value])

    Diff = layers.subtract([G_stance_out,Shared_stance_attn])
    fuse_mul = Multiply()([G_stance_out,Shared_stance_attn])
    fuse_stance = Concatenate()([G_stance_out,Shared_stance_attn,Diff,fuse_mul])
    fuse_stance_shared = Dense(100, activation="tanh")(fuse_stance)

    ############# shared specific attention for sentiment ###############
    G_sent_layer = layers.Dense(100, activation="sigmoid")
    G_sent = G_sent_layer(M)
    G_sent_layer.set_weights(try_2_weights)
    G_sent_out=Multiply()([G_sent,M])

    Shared_query = Dense(100, activation="relu")(T2_try)
    Shared_key = Dense(100, activation="relu")(M)
    Shared_value = Dense(100, activation="relu")(M)
    Shared_sent_attn = Lambda(attentionScores)([Shared_query,Shared_key,Shared_value])

    Diff = layers.subtract([G_sent_out, Shared_sent_attn])
    fuse_mul = Multiply()([G_sent_out, Shared_sent_attn])
    fuse_sent = Concatenate()([G_sent_out, Shared_sent_attn, Diff, fuse_mul])
    fuse_sent_shared = Dense(100, activation="tanh")(fuse_sent)

    stance_shared_output = Dense(2, activation="softmax", name="task_stance_shared")(fuse_stance_shared)
    senti_shared_output = Dense(3, activation="softmax", name="task_senti_shared")(fuse_sent_shared)

    model = Model([input1_stance,input2_stance,input3_stance,input1_senti,input2_senti,input3_senti],
                  [stance_specific_output,senti_specific_output,stance_shared_output,senti_shared_output])

    ############ K fold data ############
    test_feat_sequence, train_feat_sequence = total_feat_sequence[test_index], total_feat_sequence[train_index]
    test_sequence = test_feat_sequence[:, :MAX_LENGTH] 
    test_onlyfeat_sequence = test_feat_sequence[:, MAX_LENGTH:MAX_LENGTH+FEAT_LENGTH]
    test_emb_sequence = test_feat_sequence[:, -NODEEMB_LENGTH:]
    
    test_feat_sequence = tf.convert_to_tensor(test_feat_sequence.astype(np.float32))
    test_sequence = tf.convert_to_tensor(test_sequence.astype(np.float32))
    test_onlyfeat_sequence = tf.convert_to_tensor(test_onlyfeat_sequence.astype(np.float32))
    test_emb_sequence = tf.convert_to_tensor(test_emb_sequence.astype(np.float32))
    
    test_stance, train_stance = total_labels_stance[test_index], total_labels_stance[train_index]
    test_senti,train_senti = total_labels_senti[test_index], total_labels_senti[train_index]
    test_enc = np.argmax(test_stance, axis=1)
    train_enc = np.argmax(train_stance, axis=1)
    train_enc_senti = np.argmax(train_senti, axis=1)
    print("len of train", np.unique(train_enc,return_counts=True),len(train_stance))
    print("len of test", np.unique(test_enc,return_counts=True),len(test_stance))
    
    # oversample
    train_feat_sequence, train_enc, train_enc_senti = create_resample(train_feat_sequence, train_enc, train_enc_senti)
    train_feat_sequence = tf.convert_to_tensor(train_feat_sequence)
    train_sequence = train_feat_sequence[:, :MAX_LENGTH]
    train_sequence = tf.convert_to_tensor(train_sequence)
    train_onlyfeat_sequence = train_feat_sequence[:,  MAX_LENGTH:MAX_LENGTH+FEAT_LENGTH]
    train_onlyfeat_sequence = tf.convert_to_tensor(train_onlyfeat_sequence)
    train_emb_sequence = train_feat_sequence[:, -NODEEMB_LENGTH:]
    train_emb_sequence = tf.convert_to_tensor(train_emb_sequence)
    
    train_stance = to_categorical(train_enc)
    train_senti = to_categorical(train_enc_senti)

    model.compile(optimizer=Adam(0.0001),loss={'task_specific_act':'binary_crossentropy','task_specific_senti':'categorical_crossentropy','task_stance_shared':'binary_crossentropy','task_senti_shared':'categorical_crossentropy'}, loss_weights={'task_specific_act':1.0,'task_specific_senti':0.5,'task_senti_shared':0.5,'task_stance_shared':1.0}, metrics=['accuracy'])    
    print(model.summary())

    model.fit([train_sequence, train_onlyfeat_sequence, train_emb_sequence, train_sequence, train_onlyfeat_sequence, train_emb_sequence],
              [train_stance, train_senti, train_stance, train_senti], shuffle=True,validation_split=0.2,epochs=30,verbose=2)
    predicted = model.predict([test_sequence, test_onlyfeat_sequence, test_emb_sequence, 
                               test_sequence, test_onlyfeat_sequence, test_emb_sequence])

    test_enc = np.argmax(test_stance, axis=1)
    stance_pred_specific = predicted[0]
    stance_pred_shared = predicted[2]
    result_ = np.mean([stance_pred_specific,stance_pred_shared],axis=0)
    p_1 = np.argmax(result_, axis=1)
    test_accuracy = accuracy_score(test_enc, p_1)
    list_acc_stance.append(test_accuracy)
    print("test accuracy::::",test_accuracy)
    target_names = ['believe','deny']
    class_rep=classification_report(test_enc, p_1)
    print("specific confusion matrix", confusion_matrix(test_enc, p_1))
    print(class_rep)
    class_rep = classification_report(test_enc, p_1, target_names=target_names,output_dict=True)
    macro_avg = class_rep['macro avg']['f1-score']
    print("macro f1 score",macro_avg)
    list_f1_stance.append(macro_avg)

    ########### sentiment
    test_enc_senti = np.argmax(test_senti, axis=1)

    sent_pred_specific = predicted[1]
    sent_pred_shared = predicted[3]
    result_ = np.mean([sent_pred_specific,sent_pred_shared],axis=0)
    p_1 = np.argmax(result_, axis=1)
    test_accuracy = accuracy_score(test_enc_senti, p_1)
    list_acc_senti.append(test_accuracy)
    print("test accuracy::::",test_accuracy)
    target_names = ['neg', 'neu', 'pos']
    class_rep = classification_report(test_enc_senti, p_1)
    print("specific confusion matrix", confusion_matrix(test_enc_senti, p_1))
    print(class_rep)
    class_rep = classification_report(test_enc_senti, p_1, target_names=target_names, output_dict=True)
    macro_avg = class_rep['macro avg']['f1-score']
    print("macro f1 score", macro_avg)
    list_f1_senti.append(macro_avg)

print("ACCURACY :::::::::::: #############")
print("act  ::: ", list_acc_stance)
print("Mean, STD DEV", statistics.mean(list_acc_stance), statistics.stdev(list_acc_stance))

print("F1  $$$$$$$$$$$$$$$$$ ::::::::::::")
print("Specific F1 ::: ", list_f1_stance)
print("MTL Mean, STD DEV", statistics.mean(list_f1_stance), statistics.stdev(list_f1_stance))


############# sentiment 

print("ACCURACY :::::::::::: #############")
print("act  ::: ", list_acc_senti)
print("Mean, STD DEV", statistics.mean(list_acc_senti), statistics.stdev(list_acc_senti))

print("F1  $$$$$$$$$$$$$$$$$ ::::::::::::")
print("Specific F1 ::: ", list_f1_senti)
print("MTL Mean, STD DEV", statistics.mean(list_f1_senti), statistics.stdev(list_f1_senti))